In [ ]:
import sys
sys.path.append('/home/ashesh.ashesh/code/Disentangle')

In [ ]:
from IPython.display import display, HTML
import os

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(False)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
from disentangle.data_loader.patch_index_manager import TilingMode

In [ ]:
ckpt_dir = "/group/jug/ashesh/training/disentangle/2406/D25-M3-S0-L8/17"
# ckpt_dir = '/group/jug/ashesh/training/disentangle/2406/D25-M3-S0-L8/4'
outputdir = '/group/jug/ashesh/EnsDeLyon/coverage_dicts/'

image_size_for_grid_centers = 32
mmse_count = 50
elem_size = 5
custom_image_size = None
data_t_list = [0] #[0,1,2,3]
skip_percentile = 50
tiling_mode = TilingMode.ShiftBoundary

save_comparative_plots =False
enable_calibration = False
batch_size = 16
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test 
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'
assert os.path.exists(ckpt_dir)

In [ ]:
def get_dtype(ckpt_fpath):
    if os.path.isdir(ckpt_fpath):
        ckpt_fpath = ckpt_fpath[:-1] if ckpt_fpath[-1] == '/' else ckpt_fpath
    elif os.path.isfile(ckpt_fpath):
        ckpt_fpath = os.path.dirname(ckpt_fpath)
    assert ckpt_fpath[-1] != '/'
    return int(ckpt_fpath.split('/')[-2].split('-')[0][1:])

In [ ]:
dtype = get_dtype(ckpt_dir)
dtype

In [ ]:
%run ./nb_core/config_loader.ipynb

In [ ]:
tokens = ckpt_dir.split('/')
idx = tokens.index('disentangle')
if config.model.model_type == 25 and tokens[idx+1] == '2312':
    config.model.model_type = ModelType.LadderVAERestrictedReconstruction

In [ ]:
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
# from disentangle.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if 'depth3D' in config.data and config.data.depth3D > 1:
        config.data.mode_3D = True
        config.model.mode_3D = True
        
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.OnlyIba1P30
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    if 'validtarget_random_fraction' in config.data:
        config.data.validtarget_random_fraction = None
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    
    if 'dump_kth_frame_prediction' in config.training:
        config.training.dump_kth_frame_prediction = None

    if 'input_is_sum' not in config.data:
        config.data.input_is_sum = False

In [ ]:
dtype = config.data.data_type

if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
    data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
elif dtype == DataType.OptiMEM100_014:
    data_dir = f'{DATA_ROOT}/microscopy/'
elif dtype == DataType.Prevedel_EMBL:
    data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
elif dtype == DataType.AllenCellMito:
    data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
elif dtype == DataType.SeparateTiffData:
    data_dir = f'{DATA_ROOT}/ventura_gigascience'
elif dtype == DataType.SemiSupBloodVesselsEMBL:
    data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
elif dtype == DataType.Pavia2VanillaSplitting:
    data_dir = f'{DATA_ROOT}/pavia2'
# elif dtype == DataType.ExpansionMicroscopyMitoTub:
    # data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
elif dtype == DataType.ShroffMitoEr:
    data_dir = f'{DATA_ROOT}/shrofflab/'
elif dtype == DataType.HTIba1Ki67:
    data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
elif dtype == DataType.BioSR_MRC:
    data_dir = f'{DATA_ROOT}/BioSR/'
elif dtype == DataType.ExpMicroscopyV2:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v2/datafiles/'
elif dtype == DataType.TavernaSox2GolgiV2:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/acquisition2/'
elif dtype == DataType.Pavia3SeqData:
    data_dir = f'{DATA_ROOT}/pavia3_sequential/'
elif dtype == DataType.NicolaData:
    data_dir = f'{DATA_ROOT}/nikola_data/20240531/'
elif dtype == DataType.Dao3ChannelWithInput:
    data_dir = f'{DATA_ROOT}/Dao4Channel/'
elif dtype == DataType.Dao3Channel:
    data_dir = f'{DATA_ROOT}/Dao3Channel/'
elif dtype == DataType.SilvioLabCSHLData:
    data_dir = f'{DATA_ROOT}/svilen_cshl2024/'
elif dtype == DataType.ExpMicroscopyV3:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v4/405_NHS_488BODIPY/'
elif dtype == DataType.Elisa3DData:
    data_dir = f'{DATA_ROOT}/Elisa3D/'

In [ ]:
# config.model.noise_model_ch1_fpath = config.model.noise_model_ch1_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')
# config.model.noise_model_ch2_fpath = config.model.noise_model_ch2_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')

In [ ]:
%run ./nb_core/disentangle_setup.ipynb

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
idx = np.random.randint(len(val_dset))
inp_tmp, tar_tmp, *_ = val_dset[idx]
ncols = len(tar_tmp)
nrows = 2
_,ax = plt.subplots(figsize=(4*ncols,4*nrows),ncols=ncols,nrows=nrows)
for i in range(min(ncols,len(inp_tmp))):
    ax[0,i].imshow(inp_tmp[i])

for channel_id in range(ncols):
    ax[1,channel_id].imshow(tar_tmp[channel_id])

In [ ]:
if data_t_list is not None:
    val_dset.reduce_data(t_list=data_t_list)

In [ ]:
# from disentangle.analysis.calibration_coverage_v2 import patches_for_optimal_transform
# val_dset.set_img_sz(64, 32)
# model.reset_for_different_output_size(64)
# tar_patches, recons_patches = patches_for_optimal_transform(model, val_dset, num_workers=num_workers, batch_size=batch_size, mmse_count=mmse_count, elem_size=elem_size, skip_pixels=32)
def get_patches(model, dset,num_workers=4, batch_size = 32):
    dloader = DataLoader(dset, pin_memory=False, num_workers=num_workers, shuffle=False, batch_size=batch_size)
    tar_patches = []
    for batch in tqdm(dloader):
        inp, tar = batch[:2]
        inp = inp.cuda()
        tar = tar.cuda()
        tar_normalized = model.normalize_target(tar)
        tar_patches.append(tar_normalized.cpu().numpy())
    tar_patches = np.concatenate(tar_patches, axis=0)
    return tar_patches


In [ ]:
import numpy as np

tar_patches = get_patches(model, val_dset, num_workers=num_workers, batch_size=batch_size)
background_thresholds = [np.percentile(tar_patches[:,i], skip_percentile) for i in range(tar_patches.shape[1])]
print(background_thresholds)

def background_patch_detection_func(data, ch_idx):
    threshold = background_thresholds[ch_idx]
    return data.reshape(data.shape[0],-1).mean(axis=-1) < threshold


In [ ]:
# from scipy import stats
# from disentangle.analysis.calibration_coverage_v2 import find_optimal_scalar_offset
# import numpy as np

# def fit_optimal_transform(recons_patches, tar_patches, skip_percentile=50):
#     mmse_recons = recons_patches.mean(axis=1)
#     # N x C x H x W for both tar_patches and mmse_recons
#     assert tar_patches.shape == mmse_recons.shape, f'{tar_patches.shape} != {mmse_recons.shape}'
#     assert tar_patches.ndim == 4, f'{tar_patches.ndim} != 4'
#     N = tar_patches.shape[0]
#     output = {}
#     for ch_idx in range(tar_patches.shape[1]):
#         recons_data = mmse_recons[:, ch_idx]
#         tar_data = tar_patches[:, ch_idx]
#         backgrnd = np.percentile(tar_data, skip_percentile)
#         fg_mask = tar_data.mean(axis=(1,2)) > backgrnd
#         print('fg_mask', fg_mask.mean())
#         scalar, offset = find_optimal_scalar_offset(recons_data[fg_mask], tar_patches[fg_mask, ch_idx])
#         output[ch_idx] = {'scalar':scalar, 'offset':offset}
#     return output


In [ ]:
# optimal_transform_dict = fit_optimal_transform(recons_patches, tar_patches, skip_percentile=50)
# print(optimal_transform_dict)


# def get_background_thresholds():
#     return [optimal_transform_dict[0]['qval'], optimal_transform_dict[1]['qval'], optimal_transform_dict[2]['qval']]


In [ ]:
# _,ax = plt.subplots(figsize=(10,5),ncols=2)
# ax[0].imshow(tar_patches[3,2])
# ax[1].imshow(recons_patches[3,0,2])

In [ ]:
# import pickle
# # 2406/D25-M3-S0-L8/17
# fpath = '/group/jug/ashesh/data/paper_stats/Val_P64_G32_M50_Sk0/calib_training_disentangle_2406_D25-M3-S0-L8_17.pkl'
# with open(fpath, "rb") as f:
#     eval_calibration_factors = pickle.load(f)


In [ ]:
# from disentangle.analysis.stitch_prediction import stitch_predictions
# from disentangle.analysis.mmse_prediction import get_dset_predictions
# # from disentangle.analysis.stitch_prediction import get_predictions as get_dset_predictions
# def get_mean_std(nTar):
#     mean, std = val_dset.get_mean_std()
#     mean = mean['target'].squeeze()
#     std = std['target'].squeeze()
#     print(mean.shape, std.shape)
#     return mean[:nTar].reshape(1,1,1,nTar), std[:nTar].reshape(1,1,1,nTar)



# pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, val_dset,batch_size,
#                                                num_workers=num_workers,
#                                                mmse_count=mmse_count,
#                                                 model_type = config.model.model_type,
#                                               )
# tmp = np.round([x.item() for x in patch_psnr_tuple],2)
# print('Patch wise PSNR, as computed during training', tmp, np.mean(tmp))
# pred = stitch_predictions(pred_tiled, val_dset, )

# # target 
# is_list_prediction = isinstance(pred, list)
# tar_unnorm = (val_dset._data if not is_list_prediction else [val_dset.dsets[i]._data for i in range(len(val_dset.dsets))])

# if "target_idx_list" in config.data and config.data.target_idx_list is not None:
#     nTar =len(config.data.target_idx_list)
#     pred = pred[..., :len(config.data.target_idx_list)] if not is_list_prediction else [pred[i][..., :len(config.data.target_idx_list)] for i in range(len(pred))]
#     tar_unnorm = [x[...,config.data.target_idx_list] for x in tar_unnorm] if is_list_prediction else tar_unnorm[...,config.data.target_idx_list]

# nTar = pred[0].shape[-1]
# mean, std = get_mean_std(nTar)
# # normalize the target
# tar = [(x-mean)/std for x in tar_unnorm] if is_list_prediction else (tar_unnorm-mean)/std

In [ ]:
# from disentangle.analysis.calibration_coverage_v2 import find_optimal_scalar_offset
# scalar, offset = find_optimal_scalar_offset(pred, tar)
# print(scalar, offset)

In [ ]:
from disentangle.analysis.calibration_coverage import  get_calibration_coverage_data
import numpy as np
# mmse_count = 10
scalar_values = [1]
coverage_dict = {}
num_workers = 4
for scalar in scalar_values:
    var, err =get_calibration_coverage_data(model, val_dset, num_workers=num_workers,
                                       calib_stats_dict={0:{'scalar':scalar,'offset':0.0}, 
                                                                      1:{'scalar':scalar,'offset':0.0}, 
                                                                      2:{'scalar':scalar,'offset':0.0}, 
                                                                      },
                                        mmse_count=mmse_count,
                                        elem_size=elem_size,
                                        background_patch_detection_func=background_patch_detection_func)
    nan_mask = np.isnan(var).any(axis=-1).any(axis=-1)
    var = var[~nan_mask]
    err = err[~nan_mask]
    coverage_dict[scalar] = (var, err)

In [ ]:
nan_mask.shape

In [ ]:
var.shape, err.shape

In [ ]:
# def get_optimal_scaling(x,y):
#     """
#     minimize ||y - ax -b||^2
#     """
#     x = x.reshape(-1)
#     y = y.reshape(-1)
#     ux = np.mean(x)
#     uy = np.mean(y)
#     uxy = np.mean(x*y)
#     ux2 = np.mean(x*x)
#     a = (uxy - ux*uy)/(ux2 - ux*ux)
#     b = uy - a*ux
#     return a, b

In [ ]:
median_var= np.mean(var, axis=-1)
median_var.shape

In [ ]:
var.shape, err.shape

### Grid search for optimal scaling parameter.
binary search inspired.

In [ ]:
from disentangle.analysis.calibration_coverage_grid_search import grid_search
factors = []
achieved_percentiles = []
for ch_idx in range(var.shape[1]):
    print('Starting the grid search')
    factor_ch1, achieved_percentile_ch1 = grid_search(err[:,ch_idx], var[:,ch_idx], init_delta=1)
    factors.append(factor_ch1)
    achieved_percentiles.append(achieved_percentile_ch1)
    print('')


In [ ]:
x = np.linspace(0,100,100)
y = get_percentage_occurance(x, err[:,ch_idx], var[:,ch_idx], factor)

In [ ]:
plt.plot(x, y)

In [ ]:
import matplotlib.pyplot as plt
ncols = var.shape[1]
_,ax = plt.subplots(figsize=(5*ncols,5),ncols=ncols)
for col_idx in range(ncols):    
    # percentiles = x
    linestyles = [
        'solid',
        'dotted',
        'dashed',
        'dashdot',]
    
    x = np.linspace(0,100,100)
    # scaled. 
    y = get_percentage_occurance(x, err[:,col_idx], var[:,col_idx], factors[col_idx])

    ax[col_idx].plot(x, y, label=f'Scaled', linestyle= linestyles[i%len(linestyles)])
    y = get_percentage_occurance(x, err[:,col_idx], var[:,col_idx], 1)

    ax[col_idx].plot(x, y, label=f'Unscaled', linestyle= linestyles[i%len(linestyles)])

    ax[col_idx].grid()
    # facecolor to gray
    ax[col_idx].set_facecolor('lightgray')
    ax[col_idx].set_xlabel('Percentile (Confidence level)')
    ax[col_idx].set_ylabel('Percentage of data (Empirical coverage)')
# plot y=x line
    ax[col_idx].plot([0,100],[0,100], 'k--')

ax[-1].legend()

In [ ]:
# scales = []
# offsets = []
# for ch_idx in range(err.shape[1]):
#     scale,offset = get_optimal_scaling(median_var[:,ch_idx], err[:,ch_idx])
#     scales.append(scale)
#     offsets.append(offset)

# scales = np.array(scales).reshape(1,-1)
# offsets = np.array(offsets).reshape(1,-1)
# scales, offsets

In [ ]:
# np.mean((median_var*scales + offsets - err)**2)

In [ ]:
# np.mean((median_var - err)**2)

In [ ]:
# scaled_var = var*scales[...,None] + offsets[...,None]

In [ ]:
# scaled_var.shape

In [ ]:
# import scipy
# from tqdm import tqdm
# scaled_percentile_data = np.zeros(err.shape)
# unscaled_percentile_data = np.zeros(err.shape)
# for elem_idx in tqdm(range(len(scaled_var))):
#     for col_idx in range(scaled_var.shape[1]):
#         scaled_percentile_data[elem_idx,col_idx] = scipy.stats.percentileofscore(scaled_var[elem_idx,col_idx],err[elem_idx, col_idx])
#         unscaled_percentile_data[elem_idx,col_idx] = scipy.stats.percentileofscore(var[elem_idx,col_idx],err[elem_idx, col_idx])


In [ ]:
# import matplotlib.pyplot as plt
# def how_many_lie_in_k_quantiles(calibration_coverage_data, k):
#     assert k >= 0 and k <= 100
#     return np.mean(calibration_coverage_data <= k) * 100


# ncols = scaled_percentile_data.shape[1]
# _,ax = plt.subplots(figsize=(5*ncols,5),ncols=ncols)
# for col_idx in range(ncols):    
#     # percentiles = x
#     linestyles = [
#         'solid',
#         'dotted',
#         'dashed',
#         'dashdot',]
    
#     x = np.linspace(0,100,100)
#     # scaled. 
#     cov = scaled_percentile_data[:,col_idx]
#     y = [how_many_lie_in_k_quantiles(cov, k) for k in x]
#     ax[col_idx].plot(x, y, label=f'Scaled', linestyle= linestyles[i%len(linestyles)])
#     # unscaled
#     cov = unscaled_percentile_data[:,col_idx]
#     y = [how_many_lie_in_k_quantiles(cov, k) for k in x]
#     ax[col_idx].plot(x, y, label=f'Unscaled', linestyle= linestyles[i%len(linestyles)])

#     ax[col_idx].grid()
#     # facecolor to gray
#     ax[col_idx].set_facecolor('lightgray')
#     ax[col_idx].set_xlabel('Percentile (Confidence level)')
#     ax[col_idx].set_ylabel('Percentage of data (Empirical coverage)')
# # plot y=x line
#     ax[col_idx].plot([0,100],[0,100], 'k--')

# ax[-1].legend()

In [ ]:
import pandas as pd
# pd.Series(percentile_data.reshape(-1)).value_counts()

In [ ]:
# from disentangle.analysis.calibration_coverage import  get_calibration_coverage_data
# import numpy as np
# # mmse_count = 10
# def background_patch_detection_func(data, ch_idx):
#     threshold = get_background_thresholds()[ch_idx]
#     return data.reshape(data.shape[0],-1).mean(axis=-1) > threshold

# coverage_dict = {}
# cov_unscaled =get_calibration_coverage_data(model, val_dset, 
#                                        calib_stats_dict={   0:{'scalar':1.0,'offset':0.0}, 
#                                                             1:{'scalar':1.0,'offset':0.0}, 
#                                                             2:{'scalar':1.0,'offset':0.0}, 
#                                                                       },
#                                         mmse_count=mmse_count,
#                                         elem_size=elem_size,
#                                         )

# cov_scaled =get_calibration_coverage_data(model, val_dset, 
#                                        calib_stats_dict=optimal_transform_dict,
#                                         mmse_count=mmse_count,
#                                         elem_size=elem_size,
#                                         )

# coverage_dict['unscaled'] = cov_unscaled
# coverage_dict['scaled'] = cov_scaled

In [ ]:
# save the coverage_dict
import pickle
model_str = '-'.join(ckpt_dir.split('/')[-3:])
fpath = os.path.join(outputdir,f'coverage_dict_{model_str}_MMSE-{mmse_count}_E-{elem_size}_Skip-{skip_percentile}.pkl')
with open(fpath, 'wb') as f:
    pickle.dump(coverage_dict, f)
print(f'Coverage dict saved at {fpath}')

In [ ]:
import matplotlib.pyplot as plt
def how_many_lie_in_k_quantiles(calibration_coverage_data, k):
    assert k >= 0 and k <= 100
    return np.mean(calibration_coverage_data <= k) * 100


ncols = coverage_dict[list(coverage_dict.keys())[0]].shape[1]
_,ax = plt.subplots(figsize=(5*ncols,5),ncols=ncols)
for col_idx in range(ncols):    
    # percentiles = x
    linestyles = [
        'solid',
        'dotted',
        'dashed',
        'dashdot',]
    
    x = np.linspace(0,100,100)
    for i,key in enumerate(coverage_dict.keys()):
            cov = coverage_dict[key][:,col_idx]
            # skip the background patches. they correspond to nan values
            print('NaN entries fraction', np.isnan(cov).mean())
            cov = cov[~np.isnan(cov)]
            y = [how_many_lie_in_k_quantiles(cov, k) for k in x]
            ax[col_idx].plot(x, y, label=f'Scaled by:{key}', linestyle= linestyles[i%len(linestyles)])


    ax[col_idx].grid()
    # facecolor to gray
    ax[col_idx].set_facecolor('lightgray')
    ax[col_idx].set_xlabel('Percentile (Confidence level)')
    ax[col_idx].set_ylabel('Percentage of data (Empirical coverage)')
# plot y=x line
    ax[col_idx].plot([0,100],[0,100], 'k--')

ax[-1].legend()